In [ ]:
%%R
knitr::opts_chunk$set(echo = TRUE)
reticulate::use_condaenv(condaenv="/home/tds/anaconda3", required = T)
PROJHOME <- rprojroot::find_rstudio_root_file()
reticulate::py_run_string(paste0("PROJHOME='",PROJHOME,"'"))
#reticulate::r_to_py(PROJHOME)

Let's consider each intervention as a document, and therefore the corpus is the union of all interventions in a given session

In [ ]:
import re
import os

## Trying extracting Intervinients

In [ ]:
REGEX_TO_SPLIT_DOCUMENTS = "(O|A)+\s+Sr(\.|\.º|\.ª)\s+([A-zÀ-ú]|\s*)+(\(.*\))?: —"

In [ ]:
# Get the entities

deputies_and_president = set()

with open(os.path.join(PROJHOME,"resources/example_of_parlamentar_discussion/darl14sl02n014.txt")) as file:
    pattern = re.compile(REGEX_TO_SPLIT_DOCUMENTS)
    for line in file:
        match = pattern.search(line)
        if match is not None:
            deputies_and_president.add(match.group()[0:-3])

for el in sorted(deputies_and_president):
    print(el)
print(len(deputies_and_president))

## Let's try to create the list of (Intervinient, Intervention) and its mappings

In [ ]:
# Associate documents to entity

deputies_docs_unprocessed = {}
documents_unprocessed_idx = {}
documents_to_deputies = {}

doc_idx = 0
did_first_match = False

with open("resources/example_of_parlamentar_discussion/darl14sl02n014.txt") as file:
    
    first_line = next(file)
    DATE_SECTION_REGEX = "(?i)\d+ de (\w+) de \d{4}"
    romanic_number = "(?=[MDCLXVI])M*(C[MD]|D?C{0,3})(X[CL]|L?X{0,3})(I[XV]|V?I{0,3})"
    SERIES_SECTION_REGEX = romanic_number + " (Série|SÉRIE) — (Número|NÚMERO) \d{1,3}"
    
    
    pattern = re.compile(REGEX_TO_SPLIT_DOCUMENTS)
    date_section_pattern = re.compile(DATE_SECTION_REGEX)
    series_section_pattern = re.compile(SERIES_SECTION_REGEX)
    numberic_pattern = re.compile("\d+")
    
    current_docs = ""
    current_deputy = None
    
    for line in file:
        date_section_match = date_section_pattern.search(line)
        series_section_match = series_section_pattern.search(line)
        if date_section_match is not None or series_section_match is not None:
            #we are in a section, let's consume until a number appear
            line_is_page_number = False
            while not line_is_page_number:
                #check if line is number
                #if it is, then line_is_page_number = True
                line = next(file)
                numeric_match = numberic_pattern.search(line)
                if numeric_match is not None:
                    line_is_page_number = True
                    line = next(file)
        match = pattern.search(line)
        if match is not None:
            #a new document
            #is this the first one? if it is, then we already consumed the summary section
            if current_deputy is not None:
                #save current document
                documents_unprocessed_idx[doc_idx] = current_docs
                if current_deputy not in deputies_docs_unprocessed:
                    deputies_docs_unprocessed[current_deputy] = []
                deputies_docs_unprocessed[current_deputy].append(doc_idx)
                documents_to_deputies[doc_idx] = current_deputy
                doc_idx += 1
            #docs stored. start processing new one
            current_deputy = match.group()[0:-3]
            current_docs = line.replace(current_deputy, '')
        else:
            current_docs += line
    if current_deputy is not None:
                #save current document
                documents_unprocessed_idx[doc_idx] = current_docs
                if current_deputy not in deputies_docs_unprocessed:
                    deputies_docs_unprocessed[current_deputy] = []
                deputies_docs_unprocessed[current_deputy].append(doc_idx)
                documents_to_deputies[doc_idx] = current_deputy
                doc_idx += 1

                